In [1]:
# Importation des bases
import pandas as pd
import functions as fc

dict_data = fc.read_all_raw(fc.list_bases)
dict_data.keys()

dict_keys(['rejets', 'etablissements', 'emissions', 'Trait_dechets_non_dangereux', 'Trait_dechets_dangereux', 'Prod_dechets_non_dangereux', 'Prod_dechets_dangereux', 'Prelevements'])

In [2]:
etab = dict_data['etablissements']
etab_not_null = etab.dropna(subset = ['code_epsg'])
etab_null = etab.loc[etab['code_epsg'].isnull()]
gb = etab_not_null.groupby("code_epsg")
gb = [gb.get_group(x) for x in gb.groups]

In [3]:
template_search_simple = """
{
  "query": {
    "bool": {
      "should":
          { "match": { "rs_denom":   "%s"}}
            }
          }
}
"""

template_search = """
{
  "query": {
    "bool": {
      "should":
          { "match": { "rs_denom":   "%s"}},
      "filter": [
            { "prefix":  { "apet": "%s" }}
                ]
            }
          }
}
"""

rs = etab['nom_etablissement'].iloc[0]
apet = etab['code_ape'].iloc[0]

req = template_search % (rs, apet)
req_simple = template_search_simple % rs

In [4]:
etab['x'] = etab['nom_etablissement'].apply(lambda s: template_search_simple % s)

In [99]:
req = "".join(('{"index" : "sirus_2020"}\n' + etab['x']).head(2))

In [111]:
etab.head(1)

,identifiant,nom_etablissement,numero_siret,adresse,code_postal,commune,departement,region,coordonnees_x,coordonnees_y,code_epsg,code_ape,libelle_ape,code_eprtr,libelle_eprtr,geometry,x
0,9073402,STEP - BEZIERS,41003460701688,Plaine Saint Pierre,34500,BEZIERS,HERAULT,OCCITANIE,672948.91,1813634.13,27572.0,3700Z,Collecte et traitement des eaux usées,5.(f),Installations de traitement des eaux urbaines ...,POINT (672948.910 1813634.130),"\n{\n ""query"": {\n ""bool"": {\n ""shoul..."


In [19]:
df = dict_data["etablissements"]
df = df.rename({'numero_siret': "numero_siret_true"}, axis = 1)
df["code_apet"] = df["code_ape"].str[:4]

requete_type = '''{{ 
  "query": {{
    "bool": {{
      "should": [
        {{ "match": {{ "rs_denom":   "{nom_etablissement}" }}}}
      ],
      "filter": [
        {{ "match":  {{ "adr_et_post": "{code_postal}" }}}},
        {{ "prefix":  {{ "apet": "{code_apet}" }}}}
      ]
    }}
  }},
  "size": 1
}}'''




# Il est nécessaire de spécifier l'index associé à chaque requête
header = '{"index" : "sirus_2020"}'

multiple_requetes = ""

# On itère sur le dataframe d'établissements polluants pour ajouter une requête spécifique à chacun d'entre eux
n_etab = df.shape[0] # Pour l'exemple, on prend les 10 premiers

for index, row in df.iloc[0:n_etab][['nom_etablissement',  'code_postal', 'code_apet']].iterrows():
    
    multiple_requetes+= header
    multiple_requetes+= '\n'
    multiple_requetes+= requete_type.format_map(row).replace("\n","")
    multiple_requetes+= '\n'



In [20]:
res = fc.es.msearch(body = multiple_requetes, max_concurrent_searches = 500)

/home/coder/local/bin/conda/envs/basesspcloud/lib/python3.10/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


In [38]:
temp = pd.json_normalize(res,record_path=["responses"])

def get_product_echo(echo):
    if echo:
        return echo[0]['_source']
    else:
        return None
pd.DataFrame(
    temp['hits.hits'].apply(lambda echo: echo[0]['_source'] if echo else None)
)

temp.apply(
        lambda l: get_product_echo(l['hits.hits'],
                                   additional_vars=additional_vars, keyvars = keys),
        axis=1, result_type="expand")

.apply(
        lambda s: get_product_echo(s["x"]['hits']['hits'],
                                   additional_vars=additional_vars),
        axis=1, result_type="expand")

,hits.hits
0,"{'sirus_id': '243400769', 'nic': '00044', 'ape..."
1,None
2,None
3,None
4,None
...,...
8958,None
8959,"{'sirus_id': '606320190', 'nic': '00055', 'ape..."
8960,None
8961,None


In [59]:
def transform_wgs84(df, epsg):
    etab = gpd.GeoDataFrame(
        dict_data['etablissements'],
        geometry=gpd.points_from_xy(
            dict_data['etablissements']['coordonnees_x'],
            dict_data['etablissements']['coordonnees_y']
        ),
        crs = epsg)
    etab = etab.to_crs(4326)
    etab['x'] = etab['geometry'].x 
    etab['y'] = etab['geometry'].y
    etab = pd.DataFrame(etab)
    return etab

In [66]:
toto = [
    transform_wgs84(
        gb[idx],
        gb[idx]['code_epsg'].iloc[0]
    ) for idx in range(len(gb)) 
]
toto = pd.concat(
    toto
)
toto2 = pd.concat(
    [toto, etab_not_null], axis = 1
)

In [71]:
etab.shape

(8963, 16)

In [70]:
toto.head(3)

,identifiant,nom_etablissement,numero_siret,adresse,code_postal,commune,departement,region,coordonnees_x,coordonnees_y,code_epsg,code_ape,libelle_ape,code_eprtr,libelle_eprtr,geometry,x,y
0,9073402,STEP - BEZIERS,41003460701688,Plaine Saint Pierre,34500,BEZIERS,HERAULT,OCCITANIE,672948.910000,1.813634e+06,27572.0,3700Z,Collecte et traitement des eaux usées,5.(f),Installations de traitement des eaux urbaines ...,POINT (2.80299 6.40853),2.802986,6.408532
1,9076401,STEP - Pont de l'aveugle,20006710600241,17 avenue de l'adour,64600,ANGLET,PYRENEES-ATLANTIQUES,NOUVELLE-AQUITAINE,-1.498014,4.350209e+01,4326.0,3700Z,Collecte et traitement des eaux usées,5.(f),Installations de traitement des eaux urbaines ...,POINT (-1.36311 -5.98359),-1.363105,-5.983586
2,9076202,STEP - Loison Sous Lens,57202552600813,1 route de lens,62218,LOISON-SOUS-LENS,PAS-DE-CALAIS,HAUTS-DE-FRANCE,2.856548,5.043594e+01,4326.0,3700Z,Collecte et traitement des eaux usées,5.(f),Installations de traitement des eaux urbaines ...,POINT (-1.36308 -5.98354),-1.363081,-5.983541


In [12]:
# Premier essai identification etablissement

etab = dict_data['etablissements']

template_search = """
{
  "query": {
    "bool": {
      "should":
          { "match": { "rs_denom":   "%s"}},
      "filter": [
          {"geo_distance": {
                  "distance": "0.5km",
                  "location": {
                        "lat": "%s", 
                        "lon": "%s"
                              }
                            }
            }, 
            { "prefix":  { "apet": "%s" }}
                ]
            }
          }
}
"""

rs = etab['nom_etablissement'].iloc[0]
lat = etab['coordonnees_y'].iloc[0]
lon = etab['coordonnees_x'].iloc[0]
apet = etab['code_ape'].iloc[0]

template_search % (rs, lat, lon, apet)

'\n{\n  "query": {\n    "bool": {\n      "should":\n          { "match": { "rs_denom":   "STEP - BEZIERS"}},\n      "filter": [\n          {"geo_distance": {\n                  "distance": "0.5km",\n                  "location": {\n                        "lat": "1813634.13", \n                        "lon": "672948.91"\n                              }\n                            }\n            }, \n            { "prefix":  { "apet": "3700Z" }}\n                ]\n            }\n          }\n}\n'

In [5]:
specificsearch = fc.es.search(index = 'sirus_2020', body = 
'''{
  "query": {
    "bool": {
      "should":
          { "match": { "rs_denom":   "CPCU - CENTRALE DE BERCY"}},
      "filter": [
          {"geo_distance": {
                  "distance": "0.5km",
                  "location": {
                        "lat": "48.84329", 
                        "lon": "2.37396"
                              }
                            }
            }, 
            { "prefix":  { "apet": "3530" }}
                ]
            }
          }
}'''
)
specificsearch

/tmp/ipykernel_46286/4011918700.py:1: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  specificsearch = fc.es.search(index = 'sirus_2020', body =
/home/coder/local/bin/conda/envs/basesspcloud/lib/python3.10/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'took': 249,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 0.0,
  'hits': [{'_index': 'sirus_2020',
    '_type': '_doc',
    '_id': '37899345500104',
    '_score': 0.0,
    '_source': {'sirus_id': '378993455',
     'nic': '00104',
     'ape': '3530Z',
     'apet': '3530Z',
     'eff_3112_et': '151.0',
     'eff_etp_et': '141.0',
     'eff_et_effet_daaaammjj': '20181231',
     'enseigne_et1': '',
     'nom_comm_et': '',
     'adr_et_loc_geo': '7511200294',
     'adr_et_compl': '3/5/5BIS',
     'adr_et_voie_num': '3',
     'adr_et_voie_repet': '',
     'adr_et_voie_type': 'BD',
     'adr_et_voie_lib': 'DIDEROT',
     'adr_et_cedex': '',
     'adr_et_distsp': '',
     'sir_adr_et_com_lib': 'PARIS 12',
     'adr_et_post': '75012',
     'adr_et_l1': 'CLIMESPACE',
     'adr_et_l2': '',
     'adr_et_l3': '3/5/5BIS',
     'adr_et_l4': '3 BD DIDEROT',
     'adr_et_l5': '',
     'adr_e

## Les émissions de CO2

On va commencer par se focaliser sur les gros émetteurs de CO2

In [ ]:
emissions = dict_data['emissions']
emissions_co2 = emissions.loc[emissions['polluant'].str.contains("CO2")]
emissions_co2 = emissions_co2.loc[emissions_co2['polluant'].str.contains('total')]
emissions_co2.head(1)

In [ ]:
emissions_co2['quantite'].div(1000).plot.hist(logx = True)

In [ ]:
dict_data.head(1)

In [ ]:
dict_data['emissions']["annee_emission"].value_counts()

In [ ]:
dict_data['emissions']["milieu"].value_counts()

In [ ]:
dechets_dang = dict_data['Trait_dechets_dangereux']

In [ ]:
dict_data['etablissements']['identifiant'].nunique()

In [ ]:
dict_data['etablissements'].head(1)

In [ ]:
dict_data